# Text-based emotion detection (Multi-Label Text Classification)

## Abordagem Moderna para PLN: BERT pre-treinado em Português

---

### Sobre:
A tarefa se enquadra como uma "Multi-Label Text Classification", que dado uma entrada de texto consiste em classificá-la em múltiplos rótulos possíveis simultaneamente. O que é diferente da tarefa de "Multi-Class", que apesar de ter múltiplas classes consiste em classificar a entrada em apenas uma delas. A imagem abaixo exemplifica bem a diferença entre as classificações:

![multilabel_example](./images/multilabel_multiclass_example.png "https://www.mathworks.com/help/examples/nnet/win64/MultilabelImageClassificationUsingDeepLearningExample_01.png")

Os datasets utilizados nesse trabalho foram obtidos pela Trilha A do desafio [SemEval2025-Task11](https://github.com/emotion-analysis-project/SemEval2025-task11) cujo objetivo é a detecção e classificação de emoções em texto, sendo que cada texto pode expressar múltiplas emoções. As emoções contidas no datased são: raiva, nojo, medo, alegria, tristeza e surpresa. Foi utilizado o dataset em português.

Obs.: Foi instalado o NVIDIA CUDA Toolkit e a placa utilizada foi uma GTX 1070.

---

### Pre-processamento: 
O preprocessamento foi remover a coluna "id", pois não é interessante para treinar o modelo, e criar uma coluna "labels", que é um array ordenado com os valores de 1 e 0 para cada coluna das emoções respectivamente, pois para utilizar a biblioteca datasets e a trainer do huggin face espera seja esse formato.

---

### Estrategias Utilizadas: 
Após checar a distribuição das classes, como os dados são predominantemente anger e joy, foi utilizado também uma técnica para balanceamento de classes, fornecendo um maior peso para as classes restantes e resultando em um modelo mais balanceado. O tokenizer e modelo utilizados foram extraídos do huggin face [BERTimbau Base](https://huggingface.co/neuralmind/bert-base-portuguese-cased), que é um modelo BERT pre-treinado em português. Para o treino foi utilizado o train_ptbr.csv e para a avaliação foi utilizado o test_ptbr.csv. 

---

### Resultados Obtidos:
Para os resultados analisamos acurácia, f1, precisão e recall, sendo cada um deles:
- Acurácia: Proporção de previsões corretas sobre o total de exemplos.
- F1: Média entre precisão e recall, útil para avaliar o equilíbrio entre acertos e erros, especialmente em dados desbalanceados.
- Precisão: Proporção de exemplos classificados como positivos que realmente são positivos. Foca na qualidade das previsões positivas (poucos falsos positivos).
- Recall: Proporção de exemplos positivos que foram corretamente identificados pelo modelo. Foca na completude das previsões positivas (poucos falsos negativos).

Treinamos com 1, 2, 3 e 4 epochs, o melhor foi com 2 epochs, resultando em:
- Acurácia: 0.894804
- F1: 0.636481
- Precisão: 0.647028
- Recall: 0.626273

---

### Trabalhos Relacionados:
- TANG, Tiancheng; TANG, Xinhuai; YUAN, Tianyi. Fine-tuning BERT for multi-label sentiment analysis in unbalanced code-switching text. IEEE Access, v. 8, p. 193248-193256, 2020.
- ZAHERA, Hamada M. et al. Fine-tuned BERT Model for Multi-Label Tweets Classification. In: TREC. 2019. p. 1-7.
- LIU, Xuan et al. Emotion classification for short texts: an improved multi-label method. Humanities and Social Sciences Communications, v. 10, n. 1, p. 1-9, 2023.
- JABREEL, Mohammed; MORENO, Antonio. A deep learning-based approach for multi-label emotion classification in tweets. Applied Sciences, v. 9, n. 6, p. 1123, 2019.


In [78]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import datasets
import evaluate
import pandas as pd
import numpy as np
import accelerate


train_dataset = pd.read_csv('./datasets/train_ptbr.csv')
validation_dataset = pd.read_csv('./datasets/test_ptbr.csv')
train_dataset.drop(columns=['id'], inplace=True)
validation_dataset.drop(columns=['id'], inplace=True)

classes = train_dataset.columns[1:].tolist()
class2id = {cls: i for i, cls in enumerate(classes)}
id2class = {i: cls for cls, i in class2id.items()}

def create_labels(row):
    return row[classes].astype(float).tolist()

train_dataset['labels'] = train_dataset.apply(create_labels, axis=1)
validation_dataset['labels'] = validation_dataset.apply(create_labels, axis=1)

In [79]:
!nvidia-smi
print("GPU available:", torch.cuda.is_available())

Fri Jul 11 03:27:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.57                 Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1070      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   49C    P0             33W /  180W |    4428MiB /   8192MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [80]:
model_name = "neuralmind/bert-base-portuguese-cased" # google-bert/bert-base-multilingual-cased # distilbert/distilbert-base-multilingual-cased

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

train_dataset = datasets.Dataset.from_pandas(train_dataset)
validation_dataset = datasets.Dataset.from_pandas(validation_dataset)

train_tokenized_dataset = train_dataset.map(preprocess_function)
validation_tokenized_dataset = validation_dataset.map(preprocess_function)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 2226/2226 [00:00<00:00, 6084.56 examples/s]


In [81]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [82]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(classes),
    id2label=id2class,
    label2id=class2id,
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
label_matrix = np.stack(train_dataset['labels'])
class_counts = label_matrix.sum(axis=0)
class_weights = (label_matrix.shape[0] - class_counts) / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)

def custom_compute_loss(outputs, labels, num_items_in_batch=None):
    logits = outputs.logits
    loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights.to(logits.device))
    loss = loss_fct(logits, labels.float())
    return loss

In [89]:
training_args = TrainingArguments(
    
   output_dir="multilabel_emotion",
   learning_rate=2e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=2,
   weight_decay=0.01,
   eval_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=train_tokenized_dataset,
   eval_dataset=validation_tokenized_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   compute_loss_func=custom_compute_loss,
)

trainer.train()

C:\Users\mfppe\AppData\Local\Temp\ipykernel_21604\2407878682.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.378900,1.613834,0.885744,0.620020,0.606725,0.633910
2,0.371800,1.820592,0.894804,0.636481,0.647028,0.626273


TrainOutput(global_step=1484, training_loss=0.3682096743519415, metrics={'train_runtime': 226.3612, 'train_samples_per_second': 19.668, 'train_steps_per_second': 6.556, 'total_flos': 292853121878016.0, 'train_loss': 0.3682096743519415, 'epoch': 2.0})

In [90]:
text = "como paulista vi essa vergonha e fiquei chocada"

encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

predictions = outputs.logits.detach().cpu().numpy()
predictions = sigmoid(predictions)
filtered_predictions = (predictions > 0.5).astype(int).reshape(-1)

predicted_labels = [id2class[i] for i, pred in enumerate(filtered_predictions) if pred == 1]
print(f"Texto: {text}\n")
print(f"Labels predicted: {predicted_labels}\n")


predictions = predictions[0]
sorted_indices = np.argsort(predictions)[::-1]
print("Labels e suas respectivas probabilidades (desc):")
for idx in sorted_indices:
    print(f"- {id2class[idx]}: {predictions[idx]:.4f}")

Texto: como paulista vi essa vergonha e fiquei chocada

Labels predicted: ['fear', 'sadness', 'surprise']

Labels e suas respectivas probabilidades (desc):
- surprise: 0.9766
- fear: 0.8398
- sadness: 0.6884
- joy: 0.1979
- anger: 0.1459
- disgust: 0.0894
